# Statistical Analysis of Disk-Hosting Radio Galaxies: Unveiling Distribution Disparities through the Two-Sample Kolmogorov-Smirnov Test

Disk-hosting galaxies (spirals or lenticulars) that exhibit large-scale radio emission are extremely rare and under-researched. Within our broader, unpublished research project, we have collated every currently known disk-hosting radio galaxy in the Universe (n=34) together to analyse as a population. One of these analyses was the two-sample Kolmogorov-Smirnov (KS) test. This is a powerful statistical test that allows the underlying probability distributions of different datasets to be investigated and compared. When conducting this test, we aim to discern whether the currently known sample of disk-hosting radio galaxies exhibits statistical similarities or differences when compared to the extensive samples of other types of radio galaxies; ellipticals, FR-Is, FR-IIs and Seyferts. This notebook documents the process of conducting this analysis. 

## Disk-hosting galaxies vs. Ellipticals

In [3]:
# import appropriate packages
from scipy.stats import ks_2samp
import numpy as np
from astropy.table import Table
import pandas as pd

In the literature, numerous studies have generated extensive datasets of elliptical radio galaxies (as apposed to disk-hosting radio galaxies). For each of these studies, we calculated two crucial metrics, the largest linear size (LLS, in kpc) and radio power at 325MHz (in W/Hz), for the identified galaxies. These data were organised into text files, which are subsequently loaded in the following code segment. Specifically, the code extracts LLS and power values from the text files, creating separate arrays for each study. 

The next step involves concatenating the LLS and power arrays from every survey. Notably, all galaxies in each study, with the exception of 'Laing,' are categorized as Giant Radio Galaxies (GRGs), indicating that their LLS is 700 kpc or greater. Consequently, we additionally form exclusive arrays of power and size data for these GRGs.

In [18]:
# load in data
Dabhade2020_LLS,Dabhade2020_Pow = np.loadtxt('Dabhade2020GRGs.txt', delimiter='\t', unpack=True)
Dabhade2017_LLS, Dabhade2017_Pow = np.loadtxt('GRG_Dabhade_rg.txt', delimiter=',', unpack=True)
Pre1998_LLS, Pre1998_Pow = np.loadtxt('GRG_pre1998_rg.txt', delimiter=',', unpack=True)
Shoe_LLS, Shoe_Pow = np.loadtxt('GRG_Shoenmakers_rg.txt', delimiter=',', unpack=True)
Saripalli_LLS, Saripalli_Pow = np.loadtxt('GRG_Saripalli_rg.txt', delimiter=',', unpack=True)
Laing_LLS, Laing_Pow = np.loadtxt('RG_Laing_rg_bigones.txt', delimiter=',', unpack=True)
Bootes_LLS, Bootes_Pow = np.loadtxt('bootes2022GRGcalcs.txt', delimiter='\t', unpack=True, usecols=(1,6))

# power
Ell_Pow = np.concatenate((Dabhade2020_Pow,Dabhade2017_Pow,Pre1998_Pow,Shoe_Pow,Saripalli_Pow,Laing_Pow,Bootes_Pow))
GiantEll_Pow = np.concatenate((Dabhade2020_Pow,Dabhade2017_Pow,Pre1998_Pow,Shoe_Pow,Saripalli_Pow,Bootes_Pow))

# LLS
Ell_LLS = np.concatenate((Dabhade2020_LLS,Dabhade2017_LLS,Pre1998_LLS,Shoe_LLS,Saripalli_LLS,Laing_LLS,Bootes_LLS))
GiantEll_LLS = np.concatenate((Dabhade2020_LLS,Dabhade2017_LLS,Pre1998_LLS,Shoe_LLS,Saripalli_LLS,Bootes_LLS))

Similar to the elliptical galaxies, we have computed the 325MHz power and LLS of the 34 currently known disk-hosting radio galaxies. Their morphological classification— either spiral 'S' or a lenticular 'L' —is also included.

We create size and power arrays for the full population, then for the GRG, spiral and lenticular subgroups. 

In [5]:
disks = pd.read_csv('Disks.txt', delimiter=',')

# full disk population
Disk_LLS = disks['LLS'].to_numpy()
Disk_Pow = disks['P_325MHz'].to_numpy()

# GRG disks 
giantdisks = disks[disks['LLS'] >= 700]

GiantDisk_LLS = giantdisks['LLS'].to_numpy()
GiantDisk_Pow = giantdisks['P_325MHz'].to_numpy()

# spirals
spirals = disks[disks['Morphology'] == 'S']

Spirals_LLS = spirals['LLS'].to_numpy()
Spirals_Pow = spirals['P_325MHz'].to_numpy()

# lenticulars
lenticulars = disks[disks['Morphology'] == 'L']

Lenticulars_LLS = lenticulars['LLS'].to_numpy()
Lenticulars_Pow = lenticulars['P_325MHz'].to_numpy()

We now initiate the two-sample KS test on our data. The null hypothesis of this test posits that the two samples in question are drawn from the same distribution. In accordance with our predetermined significance level of 0.05, if the calculated p-values fall below this threshold, we reject the null hypothesis in favour of the alternative hypothesis, which indicates that the two samples originate from distinct underlying distributions.

Initially, we perform a one-dimensional KS test, comparing disk and elliptical samples based on a single metric—either LLS or power. The function ks_2samp facilitates this test and provides the resulting p-value for our analysis.

In [6]:
print("Test 1- LLS: Disk GRGs and Elliptical GRGs p-value =", "{:.3e}".format(ks_2samp(GiantDisk_LLS,GiantEll_LLS).pvalue))
print("Test 2- Power: Disk GRGs and Elliptical GRGs p-value =", "{:.3e}".format(ks_2samp(GiantDisk_Pow, GiantEll_Pow).pvalue))
print("Test 3- Power: Disks and Ellipticals p-value =", "{:.3e}".format(ks_2samp(Disk_Pow, Ell_Pow).pvalue))
print("Test 4- Power: Lenticulars and Spirals p-value =", "{:.3e}".format(ks_2samp(Lenticulars_Pow, Spirals_Pow).pvalue))

Test 1- LLS: Disk GRGs and Elliptical GRGs p-value = 6.286e-01
Test 2- Power: Disk GRGs and Elliptical GRGs p-value = 3.109e-07
Test 3- Power: Disks and Ellipticals p-value = 3.699e-15
Test 4- Power: Lenticulars and Spirals p-value = 4.339e-01


In test 1, we assess the LLS metric between disk and elliptical GRGs. The obtained p-value exceeds our significance level, therefore we do not reject the null hypothesis.

In test 2, where we assess the power metric between disk and elliptical GRGs, the resulting p-value falls below the significance level. Consequently, we reject the null hypothesis in favour of the alternative hypothesis.

Test 3 focuses on comparing the power metric between the entire samples of disk and elliptical galaxies. The resulting p-value is less than the significance level, prompting the rejection of the null hypothesis in favour of the alternative.

Finally, test 4, explores the potential differences between the power of lenticulars and spirals. Since the resulting p-value is greater than the significance level, we do not reject the null hypothesis in this instance.

The findings suggest a similarity between elliptical and disk populations concerning their LLS. However, a notable distinction emerges in terms of power, implying differing distributions between these two groups. On the other hand, no significant difference between lenticulars and spirals is revealed, indicating a similarity in the underlying distribution of power for these types of disk-hosting galaxies.

We now transition to the 2-dimensional KS test, which enables the assessment of our two metrics- LLS and power simultaneously. We test the potential differences in the underlying distributions between both the GRG samples and the full samples. The function utilised is retrieved from: https://github.com/syrte/ndtest.

In [7]:
import ndtest

# all disks and ellipticals
p_value_all = ndtest.ks2d2s(Ell_LLS, Ell_Pow, Disk_LLS, Disk_Pow, extra=False)
print(f"{p_value_all = :.3g}")

# disk GRGs and elliptical GRGs
p_value_giant = ndtest.ks2d2s(GiantEll_LLS, GiantEll_Pow, GiantDisk_LLS, GiantDisk_Pow, extra=False)
print(f"{p_value_giant = :.3g}")

p_value_all = 2.81e-11
p_value_giant = 9.01e-05


Both resulting p-values for these tests fall below the significance level of 0.05. We therefore reject the null hypothesis in favour of the alternative hypothesis, indicating significant differences in the underlying distributions when both LLS and power is considered. These results underscore the presence of statistically distinct characteristics between disk-hosting radio galaxies and elliptical radio galaxies. 

## Disk-hosting galaxies vs. FR-I and FR-IIs

Radio galaxies can be traditionally be classifed into FR-I and FR-II categories based on their radio morphology, with some literature proposing that the dichotomy can be formulated with the magnitude of the host galaxy and the power of the radio lobes. Our next objective is to compare our disk-hosting galaxies with these established FR-I and FR-II categories by focusing on power and magnitude metrics. 

We have acquired an FR-I and FR-II sample, however 325MHz power is not provided and must be calculated from the 178MHz flux, which is available, to enable a meaningful comparison to our disk-hosting galaxies. 

In [8]:
# load in FR-I/FR-II sample
frtable = Table.read('laing.vot')
frtable = frtable.to_pandas()
frtable = frtable.dropna(subset=['Bmag']) # remove rows with no magnitude value

# convert to 178MHz flux to 325MHz flux assuming a radio spectral index of -0.7
frtable['S325'] = (325 / 178) ** (-0.7) * frtable['S178']

Now equipped with the 325MHz flux, we proceed to compute the 325MHz power using the formula outlined in https://iopscience.iop.org/article/10.1088/2041-8205/714/2/L190. This calculation necessitates the determination of the luminosity distance, given in metres. To calculate this key value, we develop a dedicated function utilising functions from the astropy package. 

In [9]:
from astropy.cosmology import FlatLambdaCDM

def calculate_DL_Mpc(z):
    """Luminosity distance (DL) calculator. Result is in Mpc. 
       z: redshift
    """
    cosmo = FlatLambdaCDM(H0=69.6, Om0=0.286)
    DL_Mpc = cosmo.luminosity_distance(z).to_value()
    return DL_Mpc

frtable['DL_Mpc'] = frtable['z'].apply(calculate_DL_Mpc)

frtable['DL_m'] = frtable['DL_Mpc'] * 3.086e+22 # convert DL from Mpc to m

# calculate the 325MHz power
frtable['P_325MHz'] = 4 * np.pi * frtable['DL_m']**2 / (1 + frtable['z'])**1.7 * frtable['S325'] * 1e-26

Next, we partition the FR-I and FR-II galaxies into separate data sets, then create arrays of 325MHz power and magnitude for each class. 

In [10]:
fr_i_df = frtable[frtable['FR'] == 'I']
fr_ii_df = frtable[frtable['FR'] == 'II']

FRI_Pow = np.array(fr_i_df['P_325MHz'])
FRI_Mag = np.array(fr_i_df['Bmag'])
FRII_Pow = np.array(fr_ii_df['P_325MHz'])
FRII_Mag = np.array(fr_ii_df['Bmag'])

Within our previously loaded data set of disk-hosting radio galaxies, magnitudes are provided for most of the galaxies. We extract both the magnitudes and powers of this sample into their respective arrays.   

In [11]:
disks_subset = disks[~disks['Bmag'].isna()]

Disk_Mag = disks_subset['Bmag'].to_numpy()
Diskswithmag_Pow = disks_subset['P_325MHz'].to_numpy()

We now proceed with the 2-dimensional KS test to compare the power and magnitude of disk-hosting galaxies to both FR-Is and FR-IIs. 

In [12]:
# disks and FR-Is
p_value_frI = ndtest.ks2d2s(FRI_Pow, FRI_Mag, Diskswithmag_Pow, Disk_Mag, extra=False)
print(f"{p_value_frI = :.3g}")

# disks and FR-IIs
p_value_frII = ndtest.ks2d2s(FRII_Pow, FRII_Mag, Diskswithmag_Pow, Disk_Mag, extra=False)
print(f"{p_value_frII = :.3g}")

p_value_frI = 0.00026
p_value_frII = 3.03e-11


Similar to the 2-dimensional test between disks and ellipticals, both the resulting p-values for the tests between disks and FR-Is and FR-IIs fall below the significance level of 0.05. We therefore reject the null hypothesis in favour of the alternative hypothesis. This indicates that disk-hosting radio galaxies have statistically distinct characteristics to FR-I and FR-II galaxies.

## Disk-hosting galaxies vs. Seyferts

Seyfert galaxies constitute another type of radio galaxy and are typically hosted by spirals. To compare these galaxies to our disk-hosting galaxies, we first obtained a catalog of Seyferts, presented in the files 'sycata1.vot' and 'sycata2.vot'. The first file contains the NVSS name and ID for each galaxy, while the second table includes the ID, redshift and the luminosity distance. We require information from both of these files, and therefore merge them together based on their ID.  

In [13]:
seyferttable1 = Table.read('sycata1.vot').to_pandas()
seyferttable2 = Table.read('sycata2.vot').to_pandas()
seyferts = pd.merge(seyferttable1, seyferttable2, on="ID")

Additionally, we need the 325MHz power for these Seyfert galaxies. Leveraging the NVSS catalog of radio emissions, which contains 1400MHz flux data alongside NVSS names, we can facilitate a cross-match process. This cross-match allows us to integrate the 1400MHz flux values from the NVSS catalog into the Seyfert catalog.

In [14]:
nvss = Table.read('nvsscatalog.vot').to_pandas()

# making the NVSS names from the seyferts catalog compatible with those from the NVSS catalog
seyferts['NVSS'].replace('', pd.NA, inplace=True)
seyferts = seyferts.dropna(subset=['NVSS'])
seyferts['NVSS'] = seyferts['NVSS'].str[5:]

# cross-matching seyferts and NVSS catalog
seyferts = pd.merge(seyferts, nvss, on="NVSS")

Subsequently, we follow a similar method as employed above for the FR-I/FR-IIs to convert the 1400MHz flux into 325MHz flux, and further into 325MHz power. 

In [15]:
seyferts['S325'] = (325 / 1400) ** (-0.7) * seyferts['S1.4'] # assuming alpha = -0.7
seyferts['DL_m'] = seyferts['LD'] * 3.086e+22 # converting luminosity distance from Mpc to m
seyferts['P_325MHz'] = 4 * np.pi * seyferts['DL_m']**2 / (1 + seyferts['z'])**1.7 * seyferts['S325'] * 1e-29

Seyfert_Pow = seyferts['P_325MHz'] 

With these 325MHz power values, we can proceed to conduct a one-dimensional KS test to compare the powers of Seyferts and disk-hosting radio galaxies. We do not compare LLS values as Seyferts are categorically much smaller. 

In [16]:
print("Power: Disks and Seyferts p-value =", "{:.3e}".format(ks_2samp(Disk_Pow, Seyfert_Pow).pvalue))

Power: Disks and Seyferts p-value = 1.120e-12


Again, we find that the resulting p-value for this test between disks and Seyferts falls below the significance level of 0.05, thus we reject the null hypothesis in favour of the alternative hypothesis. This suggests that disk-hosting radio galaxies exhibit statistically significant differences in power when compared to Seyfert galaxies.

In summary, our analysis of the currently known sample of disk-hosting radio galaxies has revealed that this rare and novel class of radio galaxy exhibits statistical differences when compared to the extensive samples of other types of radio galaxies, namely ellipticals, FR-Is, FR-IIs and Seyferts. These findings contribute valuable insights into the unique properties and underlying distributions of disk-hosting radio galaxies in contrast to other radio galaxy classifications.